# Test sampling algorithms (dynamic nested)

In [ ]:
import os
import time

import numpy as np
import psdist as ps
import psdist.plot as psv
import scipy.interpolate
import ultraplot as plt
from tqdm import tqdm

import ment

In [ ]:
plt.rc["cmap.discrete"] = False
plt.rc["cmap.sequential"] = "viridis"
plt.rc["figure.facecolor"] = "white"
plt.rc["grid"] = False

Create a particle distribution.

In [ ]:
# Settings
ndim = 2
size = 1_000_000
n_modes = 5
seed = 1241

# Create gaussian particle distribution
rng = np.random.default_rng(seed)
mean = np.zeros(ndim)
cov = np.identity(ndim)
for i in range(ndim):
    for j in range(i):
        cov[i, j] = cov[j, i] = rng.uniform(-0.4, 0.4)
x = rng.multivariate_normal(mean, cov, size=size)

# Add gaussian blobs
for _ in range(n_modes):
    scale = rng.uniform(0.8, 1.5, size=ndim)
    loc = rng.uniform(-5.0, 3.0, size=ndim)
    x = np.vstack([x, rng.normal(loc=loc, scale=scale, size=(size // n_modes, ndim))])
x = x - np.mean(x, axis=0)

rng.shuffle(x)

# Unit std
x = x / np.std(x, axis=0)

x_true = np.copy(x)

In [ ]:
limits = ps.limits(x_true)

grid = psv.CornerGrid(ndim, corner=True, figwidth=(1.25 * ndim))
grid.plot(x_true, bins=50, limits=limits, cmap="mono")
plt.show()

Compute ground-truth histogram. This will act as our ground-truth distribution function to sample from.

In [ ]:
n_bins = 25
values, edges = np.histogramdd(x_true, bins=n_bins, range=limits, density=True)
hist = ps.Histogram(values=values, edges=edges)
print("hist.shape =", hist.shape)

Interpolate to obtain a smooth density function.

In [ ]:
prob_func = scipy.interpolate.RegularGridInterpolator(
    hist.coords,
    hist.values,
    method="linear",
    bounds_error=False,
    fill_value=0.0,
)

## Plot utilities

In [ ]:
def plot_corner_samp(x_samp: np.ndarray):
    limits = ps.limits(x_true, rms=2.5)

    grid = psv.CornerGrid(ndim, corner=True, figwidth=(1.25 * ndim))
    grid.set_limits(limits)
    grid.plot_hist(hist, cmap="mono")
    grid.plot(x_samp[:, :], kind="hist", alpha=0.0, diag_kws=dict(color="red"))
    grid.plot(
        x_samp[:1000, :],
        diag=False,
        kind="scatter",
        color="red",
        s=0.25,
    )
    return grid

## Nested sampler

In [ ]:
import dynesty
import math

In [ ]:
# # defining constants
# r = 2.0  # radius
# w = 0.1  # width
# c1 = np.array([-3.5, 0.0])  # center of shell 1
# c2 = np.array([3.5, 0.0])  # center of shell 2
# const = math.log(1.0 / math.sqrt(2.0 * math.pi * w**2))  # normalization constant


# # log-likelihood of a single shell
# def logcirc(theta, c):
#     d = np.sqrt(np.sum((theta - c) ** 2, axis=-1))  # |theta - c|
#     return const - (d - r) ** 2 / (2.0 * w**2)


# # log-likelihood of two shells
# def log_prob_func(theta):
#     log_prob = np.logaddexp(logcirc(theta, c1), logcirc(theta, c2))
#     return log_prob

# # our prior transform
# def prior_transform(x):
#     return 12.0 * x - 6.0

In [ ]:
def log_prob_func(x):
    return np.squeeze(prob_func(x) + 1.00e-15)
    
def prior_transform(z):
    return (z - 0.5) * 4.0

In [ ]:
rng = np.random.default_rng(None)

sampler = dynesty.DynamicNestedSampler(
    log_prob_func,
    prior_transform,
    ndim=ndim,
    rstate=rng,
    sample="rwalk",
    bound="balls",
)

In [ ]:
sampler.run_nested()

In [ ]:
x_samp = sampler.results.samples
rng.shuffle(x_samp)
psv.plot_corner(
    x_samp[:, :], 
    bins=100, 
    mask=False, 
    # limits=ps.limits(x_samp, rms=4.0)
)

In [ ]:
print(x_samp.shape)
rng.shuffle(x_samp)
grid = plot_corner_samp(x_samp)

In [ ]:
help(sampler)

In [ ]:
help(sampler.run_nested)